In [4]:
#!/usr/bin/env python3
"""
Read the link-budget CSV, build a small 'library' (Parameter/python_variable/Units/Formula),
and print results for the 'Endurosat X-band' column.

Usage:
    python print_endurosat_xband.py [/path/to/x_down_info.csv]

If no path is given, defaults to /mnt/data/x_down_info.csv
"""

import sys
import math
import pandas as pd

TARGET_COLUMN = "Endurosat X-band"

def to_float(x):
    try:
        # pandas may already give us float; if it's a string like "1.38E-23" this still works
        return float(x)
    except Exception:
        return None

def guess_output_units(units_in, formula):
    # If the formula is a log10 ratio (10*log10(...) or 20*log10(...)) we assume dB output
    if isinstance(formula, str) and "log10" in formula:
        return "dB"
    return units_in if isinstance(units_in, str) else ""

def fmt_num(x):
    if x is None:
        return "N/A"
    # tidy numeric formatting
    if abs(x) < 1e-3 or abs(x) >= 1e4:
        return f"{x:.3e}"
    # show up to 3 sig figs, strip trailing zeros
    s = f"{x:.3g}"
    # ensure something like "3.00" becomes "3"
    try:
        xi = float(s)
        if abs(xi - round(xi)) < 1e-9:
            return str(int(round(xi)))
    except Exception:
        pass
    return s

def main(csv_path: str):
    df = pd.read_csv(csv_path)

    # Build the "library" structure and a variable -> value map for the target column
    lib = []
    values = {}

    required_cols = {"Parameter", "python_variable", "Units", "Formula", TARGET_COLUMN}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"CSV is missing required columns: {', '.join(sorted(missing))}")

    for _, row in df.iterrows():
        param = row["Parameter"]
        var   = row["python_variable"]
        units = row["Units"]
        formula = row["Formula"]
        raw_val = row[TARGET_COLUMN]

        entry = {
            "Parameter": param,
            "python_variable": var,
            "Units": units,
            "Formula": formula if isinstance(formula, str) and formula.strip() else None,
        }
        lib.append(entry)

        # seed values from the target column (numeric only)
        val_f = to_float(raw_val)
        if isinstance(var, str) and var and val_f is not None:
            values[var] = val_f

    # Evaluate formulas where present, using values (which include Endurosat X-band numbers)
    # Put 'math' in the environment; keep builtins disabled.
    env_globals = {"__builtins__": {}}
    env_locals = {"math": math}
    env_locals.update(values)  # make all known variables available

    results = {}  # python_variable -> computed value (or raw if no formula)
    for entry in lib:
        var = entry["python_variable"]
        param = entry["Parameter"]
        units = entry["Units"]
        formula = entry["Formula"]

        # start from raw value if present
        raw = values.get(var, None)

        if formula:
            try:
                val = eval(formula, env_globals, env_locals)
                # store and also update locals so downstream formulas can depend on earlier results
                results[var] = val
                env_locals[var] = val
            except Exception:
                # if the formula can't be evaluated, fall back to the raw value from the column
                results[var] = raw
        else:
            results[var] = raw

    # Print: "Parameter: value [units]"
    for entry in lib:
        param = entry["Parameter"]
        var   = entry["python_variable"]
        units_in = entry["Units"]
        formula  = entry["Formula"]

        val = results.get(var, None)
        units_out = guess_output_units(units_in, formula)
        units_str = f" {units_out}" if units_out else ""

        print(f"{param}: {fmt_num(val)}{units_str}")

if __name__ == "__main__":
    csv_path = sys.argv[1] if len(sys.argv) > 1 else "/mnt/data/x_down_info.csv"
    main(csv_path)


OSError: [Errno 22] Invalid argument: '--f=c:\\Users\\WilliamAvison\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3c47f55ff71d6f46d6f9e7071bc96acda94032485.json'

In [6]:
#!/usr/bin/env python3
"""
Read the link-budget CSV, build a small 'library' (Parameter/python_variable/Units/Formula),
and print results for the 'Endurosat X-band' column.

Usage (terminal):
    python print_endurosat_xband.py /path/to/x_down_info.csv

Usage (notebook):
    # just run this cell; it will auto-pick a local CSV if available,
    # otherwise fall back to /mnt/data/x_down_info.csv if present.
"""

import os
import math
import argparse
import pandas as pd

TARGET_COLUMN = "Endurosat X-band"

def to_float(x):
    try:
        return float(x)
    except Exception:
        return None

def guess_output_units(units_in, formula):
    if isinstance(formula, str) and "log10" in formula:
        return "dB"
    return units_in if isinstance(units_in, str) else ""

def fmt_num(x):
    if x is None:
        return "N/A"
    if abs(x) < 1e-3 or abs(x) >= 1e4:
        return f"{x:.3e}"
    s = f"{x:.3g}"
    try:
        xi = float(s)
        if abs(xi - round(xi)) < 1e-9:
            return str(int(round(xi)))
    except Exception:
        pass
    return s

def resolve_csv_path(cli_csv: str | None):
    # If user provided a path and it exists, use it.
    if cli_csv and cli_csv.lower().endswith(".csv") and os.path.isfile(cli_csv):
        return cli_csv
    # Otherwise try a few sensible defaults.
    candidates = [
        "./x_down_info.csv",
        os.path.expanduser("~/Downloads/x_down_info.csv"),
        "/mnt/data/x_down_info.csv",
    ]
    for p in candidates:
        if os.path.isfile(p):
            return p
    return None

def main(csv_path: str):
    df = pd.read_csv(csv_path)

    required_cols = {"Parameter", "python_variable", "Units", "Formula", TARGET_COLUMN}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"CSV is missing required columns: {', '.join(sorted(missing))}")

    # Build the library + seed values from the target column
    lib = []
    values = {}
    for _, row in df.iterrows():
        entry = {
            "Parameter": row["Parameter"],
            "python_variable": row["python_variable"],
            "Units": row["Units"],
            "Formula": row["Formula"] if isinstance(row["Formula"], str) and row["Formula"].strip() else None,
        }
        lib.append(entry)

        var = entry["python_variable"]
        raw_val = row[TARGET_COLUMN]
        val_f = to_float(raw_val)
        if isinstance(var, str) and var and val_f is not None:
            values[var] = val_f

    # Safe-ish eval env
    env_globals = {"__builtins__": {}}
    env_locals = {"math": math}
    env_locals.update(values)

    results = {}
    for entry in lib:
        var = entry["python_variable"]
        formula = entry["Formula"]
        raw = values.get(var, None)
        if formula:
            try:
                val = eval(formula, env_globals, env_locals)
                results[var] = val
                env_locals[var] = val
            except Exception:
                results[var] = raw
        else:
            results[var] = raw

    # Print "Parameter: value units"
    for entry in lib:
        param = entry["Parameter"]
        var = entry["python_variable"]
        units_in = entry["Units"]
        formula = entry["Formula"]
        val = results.get(var, None)
        units_out = guess_output_units(units_in, formula)
        units_str = f" {units_out}" if units_out else ""
        print(f"{param}: {fmt_num(val)}{units_str}")

if __name__ == "__main__":
    # Robust argument parsing (ignores Jupyter's --f=... argument)
    parser = argparse.ArgumentParser(add_help=True)
    parser.add_argument("csv", nargs="?", help="Path to the CSV file")
    args, _ = parser.parse_known_args()
    csv_path = resolve_csv_path(args.csv)

    if not csv_path:
        raise SystemExit(
            "Could not find a CSV. Pass a path explicitly, e.g.\n"
            "  python print_endurosat_xband.py C:\\path\\to\\x_down_info.csv\n"
            "or place x_down_info.csv in the current folder."
        )

    main(csv_path)


X-Band lower limit (downlink): 8.03 GHz
X-Band upper limit (downlink): 8.4 GHz
Selected Carrier Frequency: 8.22 GHz
Wavelength: nan m
Tx Power: 3.01 dB
Peak Boresight Antenna Gain: 17 dBi
Interconnect losses (negative) / gains (positive): 0.5 dB
Calculated EIRP: nan W
Orbit altitude: 500 km
Elevation ground to sat: 10 degrees
Calculated slant range: 1690 km
Free Space Path Loss term: -175 dB
Atmospheric attenuation: 1 dB
Rain attenuation ('fade'): 0.5 dB
Scintillation loss: 1 dB
Polarisation loss: 1 dB
Other propagation losses: 2 dB
Total Propagation Loss: -170 dB
System symbol rate (or bandwidth): 30 MSps (or MHz)
MODCOD scheme: N/A
MODCOD implementation loss: 1.9 dB
Spectral efficiency: 3 bps/Hz
Other radio/protocol overheads: N/A (%)
Channel Throughput: 89 Mbps
CODMOD-inferred required Eb/N0 (inc. implementation losses): 7.39 dB
Isotropically received power at satellite antenna input: nan dBW
G/T: 24.4 dB/K
Dish diameter: 3.7 m
Calculated HPBW: 0.69 degrees
Dish pointing error: 0.1 

In [8]:
#!/usr/bin/env python3
"""
Compute Endurosat X-band link margin vs. elevation for two altitudes and
export both a PNG plot and a CSV table.

Inputs pulled from CSV (Endurosat X-band column):
- selected_carrier_frequency [GHz]
- tx_power [W]
- peak_boresight_antenna_gain [dBi]
- interconnect_losses_gains [dB]  (positive = loss)
- atmospheric_attenuation, rain_attenuation, scintillation_loss,
  polarisation_loss, other_propagation_losses [dB]
- g_t [dB/K]
- data_rate_in_db_hz [dB-Hz]
- codmod_inferred_required_eb_n0 [dB]
- other_overall_link_losses, gain_rolloff_loss_at_pointing_error [dB]

Outputs:
- ./Endurosat_X-band_link_margin_vs_elevation.csv
- ./Endurosat_X-band_link_margin_vs_elevation.png
"""

import os
import math
import argparse
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

EARTH_RADIUS_KM = 6371.0
TARGET_COLUMN = "Endurosat X-band"

# -------- helpers --------
def to_float(x):
    try:
        return float(str(x).replace("%", "").strip())
    except Exception:
        return None

def resolve_csv_path(cli_csv: str | None):
    if cli_csv and cli_csv.lower().endswith(".csv") and Path(cli_csv).is_file():
        return cli_csv
    for p in ["./x_down_info.csv",
              os.path.expanduser("~/Downloads/x_down_info.csv"),
              "/mnt/data/x_down_info.csv"]:
        if Path(p).is_file():
            return p
    raise FileNotFoundError(
        "CSV not found. Pass a path explicitly (e.g. --csv C:\\path\\x_down_info.csv) "
        "or place x_down_info.csv in the current folder."
    )

def get_var(df: pd.DataFrame, varname: str, default=None):
    s = df.loc[df["python_variable"] == varname, TARGET_COLUMN]
    return to_float(s.iloc[0]) if not s.empty else default

def slant_range_km(alt_km: float, elev_deg: float) -> float:
    e = math.radians(elev_deg)
    R = EARTH_RADIUS_KM
    H = R + alt_km
    return -R * math.sin(e) + math.sqrt((R * math.sin(e)) ** 2 + H ** 2 - R ** 2)

def fspl_db(range_km: float, freq_ghz: float) -> float:
    # Standard: 92.45 + 20log10(R[km]) + 20log10(f[GHz])
    return 92.45 + 20 * math.log10(range_km) + 20 * math.log10(freq_ghz)

def eirp_dbw(P_w: float, G_tx_dbi: float, L_tx_db: float) -> float:
    return 10 * math.log10(P_w) + (G_tx_dbi or 0.0) - (L_tx_db or 0.0)

def cn0_dbhz(eirp_dbw: float, L_total_db: float, G_T_dbk: float,
             k_val: float = 1.38e-23, extra_losses_db: float = 0.0) -> float:
    k_db = 10 * math.log10(k_val)
    return eirp_dbw - L_total_db + (G_T_dbk or 0.0) - k_db - (extra_losses_db or 0.0)

# -------- main compute --------
def compute_table(csv_path: str,
                  altitudes=(500, 600),
                  elevations=(0, 5, 10, 15, 20)) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    # constants from CSV
    f_ghz = get_var(df, "selected_carrier_frequency")
    tx_power_w = get_var(df, "tx_power")
    Gtx_dbi = get_var(df, "peak_boresight_antenna_gain")
    Ltx_db = get_var(df, "interconnect_losses_gains", 0.0)

    atm = get_var(df, "atmospheric_attenuation", 0.0)
    rain = get_var(df, "rain_attenuation", 0.0)
    scint = get_var(df, "scintillation_loss", 0.0)
    pol = get_var(df, "polarisation_loss", 0.0)
    other_prop = get_var(df, "other_propagation_losses", 0.0)

    G_T = get_var(df, "g_t")
    Rb_dBHz = get_var(df, "data_rate_in_db_hz")
    EbN0_req = get_var(df, "codmod_inferred_required_eb_n0", 0.0)
    other_overall = get_var(df, "other_overall_link_losses", 0.0)
    rolloff = get_var(df, "gain_rolloff_loss_at_pointing_error", 0.0)

    rows = []
    for alt in altitudes:
        for el in elevations:
            sr_km = slant_range_km(alt, el)
            Lfs = fspl_db(sr_km, f_ghz)
            Ltotal = Lfs + atm + rain + scint + pol + other_prop
            eirp = eirp_dbw(tx_power_w, Gtx_dbi, Ltx_db)
            cn0 = cn0_dbhz(eirp, Ltotal, G_T, 1.38e-23, other_overall + rolloff)
            ebn0 = cn0 - Rb_dBHz
            margin = ebn0 - EbN0_req

            rows.append({
                "orbit_altitude_km": alt,
                "elevation_deg": el,
                "slant_range_km": sr_km,
                "fspl_db": Lfs,
                "total_prop_loss_db": Ltotal,
                "eirp_dbw": eirp,
                "cn0_dbhz": cn0,
                "ebn0_db": ebn0,
                "link_margin_db": margin,
            })

    return pd.DataFrame(rows)

def make_plot(df_out: pd.DataFrame, png_path: str):
    plt.figure(figsize=(8, 6))
    for alt in sorted(df_out["orbit_altitude_km"].unique()):
        subset = df_out[df_out["orbit_altitude_km"] == alt].sort_values("elevation_deg")
        plt.plot(subset["elevation_deg"], subset["link_margin_db"], marker="o", label=f"{int(alt)} km")
    plt.title("Link Margin vs Elevation for Endurosat X-band")
    plt.xlabel("Elevation (degrees)")
    plt.ylabel("Link Margin (dB)")
    plt.xticks(sorted(df_out["elevation_deg"].unique()))
    plt.grid(True)
    plt.legend()
    plt.savefig(png_path, bbox_inches="tight")
    plt.close()

def main():
    parser = argparse.ArgumentParser(description="Generate Link Margin vs Elevation plot and CSV.")
    parser.add_argument("--csv", help="Path to x_down_info.csv")
    parser.add_argument("--out-csv", default="Endurosat_X-band_link_margin_vs_elevation.csv")
    parser.add_argument("--out-png", default="Endurosat_X-band_link_margin_vs_elevation.png")
    parser.add_argument("--alts", nargs="*", type=float, default=[500, 600],
                        help="Altitudes in km (space-separated)")
    parser.add_argument("--elevs", nargs="*", type=float, default=[0, 5, 10, 15, 20],
                        help="Elevations in degrees (space-separated)")

    args, _ = parser.parse_known_args()
    csv_path = resolve_csv_path(args.csv)

    df_out = compute_table(csv_path, altitudes=args.alts, elevations=args.elevs)
    df_out.to_csv(args.out_csv, index=False)
    make_plot(df_out, args.out_png)

    print(f"Wrote CSV -> {args.out_csv}")
    print(f"Wrote PNG -> {args.out_png}")

if __name__ == "__main__":
    main()


Wrote CSV -> Endurosat_X-band_link_margin_vs_elevation.csv
Wrote PNG -> Endurosat_X-band_link_margin_vs_elevation.png


In [10]:
#!/usr/bin/env python3
"""
Notebook/CLI friendly pass data budget generator.

- If run with CLI args, behaves like before.
- If run with *no* args (e.g., pasted into a Jupyter cell), it:
  * looks for Endurosat_X-band_link_margin_vs_elevation.csv and x_down_info.csv
    in ./ or /mnt/data
  * finds all altitudes in the link CSV
  * writes pass_data_budget_<alt>km.csv for each altitude
"""

import argparse, math, sys, os
from pathlib import Path
import pandas as pd

MU_EARTH = 398600.4418  # km^3/s^2
R_EARTH = 6371.0        # km

def find_file(name):
    for p in [Path(name), Path("/mnt/data")/name]:
        if p.is_file():
            return str(p)
    return None

def get_supported_rate(info_csv: str):
    df = pd.read_csv(info_csv)
    s = df.loc[df["python_variable"].astype(str)=="supported_user_data_rate"]
    if s.empty:
        s = df[df["Parameter"].astype(str).str.contains("Supported user data rate", case=False, na=False)]
    if s.empty:
        raise ValueError("Could not find 'Supported user data rate' in x_down_info.csv")
    val = s.iloc[0]["Endurosat X-band"]
    units = s.iloc[0]["Units"] if "Units" in df.columns else ""
    rate_val = float(str(val).strip())
    unit_str = (units or "").lower()
    if "mb" in unit_str:  bps, pretty = rate_val*1e6, "Mb/s"
    elif "kb" in unit_str: bps, pretty = rate_val*1e3, "kb/s"
    elif "b/s" in unit_str or "bps" in unit_str: bps, pretty = rate_val, "b/s"
    else: bps, pretty = rate_val*1e6, "Mb/s (assumed)"
    return bps, pretty

def central_angle_from_slant(range_km: float, alt_km: float) -> float:
    H = R_EARTH + alt_km
    rho = range_km
    cos_theta = (R_EARTH*R_EARTH + H*H - rho*rho) / (2*R_EARTH*H)
    return math.acos(max(-1.0, min(1.0, cos_theta)))

def mean_motion_rad_s(alt_km: float) -> float:
    a = R_EARTH + alt_km
    return math.sqrt(MU_EARTH / (a*a*a))

def build_budget(link_csv: str, info_csv: str, alt_km: float,
                 emin: float | None, margin_min: float, out_csv: str) -> pd.DataFrame:
    df = pd.read_csv(link_csv)
    need = {"orbit_altitude_km","elevation_deg","link_margin_db","slant_range_km"}
    if not need.issubset(df.columns):
        raise ValueError(f"link CSV missing columns: {need - set(df.columns)}")
    dfa = df[df["orbit_altitude_km"].astype(float)==float(alt_km)].copy()
    if dfa.empty:
        raise ValueError(f"No rows for orbit_altitude_km={alt_km}")
    dfa.sort_values("elevation_deg", inplace=True)
    bps, unit_str = get_supported_rate(info_csv)
    if emin is None:
        emin = float(dfa["elevation_deg"].min())

    e = dfa["elevation_deg"].to_list()
    m = dfa["link_margin_db"].to_list()
    r = dfa["slant_range_km"].to_list()
    theta = [central_angle_from_slant(x, alt_km) for x in r]
    n = mean_motion_rad_s(alt_km)

    rows = []
    for i in range(len(e)-1):
        e0, e1 = e[i], e[i+1]
        m0, m1 = m[i], m[i+1]
        th0, th1 = theta[i], theta[i+1]
        if max(e0,e1) < emin: continue
        e_start, e_end = max(min(e0,e1), emin), max(e0,e1)
        if e_end <= e_start: continue

        def lerp(x0,y0,x1,y1,x):
            t=(x-x0)/(x1-x0); return y0+t*(y1-y0)

        if e1 < e0:  # ensure ascending
            e0,e1 = e1,e0; m0,m1 = m1,m0; th0,th1 = th1,th0

        th_start = lerp(e0,th0,e1,th1,e_start)
        th_end   = lerp(e0,th0,e1,th1,e_end)
        m_start  = lerp(e0,m0, e1,m1, e_start)
        m_end    = lerp(e0,m0, e1,m1, e_end)

        dtheta = max(0.0, th_start - th_end)
        bin_time_s = 2.0 * dtheta / n  # rise+set

        usable_fraction = 1.0
        if (m_start < margin_min) and (m_end < margin_min):
            usable_fraction = 0.0
        elif (m_start < margin_min) or (m_end < margin_min):
            if m_end != m_start:
                e_cross = e_start + (margin_min - m_start) * (e_end - e_start) / (m_end - m_start)
                usable_fraction = (e_end - e_cross) / (e_end - e_start) if e_start <= e_cross <= e_end else (1.0 if m_end >= margin_min else 0.0)
            else:
                usable_fraction = 0.0 if m_start < margin_min else 1.0

        data_bits = bps * bin_time_s * usable_fraction
        data_mib  = data_bits / (1024*1024*8)

        rows.append({
            "alt_km": float(alt_km),
            "elev_start_deg": float(e_start),
            "elev_end_deg": float(e_end),
            "theta_start_deg": math.degrees(th_start),
            "theta_end_deg": math.degrees(th_end),
            "bin_time_s": bin_time_s,
            "margin_start_db": m_start,
            "margin_end_db": m_end,
            "usable_fraction": usable_fraction,
            "data_bits": data_bits,
            "data_MiB": data_mib,
            "rate_bps": bps,
            "rate_unit": unit_str,
        })

    out = pd.DataFrame(rows)
    out.to_csv(out_csv, index=False)
    print(f"[{int(alt_km)} km] time≥thr: {out['bin_time_s'].sum():.1f}s  data: {out['data_MiB'].sum():.1f} MiB")
    return out

def main():
    parser = argparse.ArgumentParser(description="Create data budget for a pass from two CSVs.")
    parser.add_argument("--link-csv", required=False, help="Path to Endurosat_X-band_link_margin_vs_elevation.csv")
    parser.add_argument("--info-csv", required=False, help="Path to x_down_info.csv")
    parser.add_argument("--alt-km", type=float, required=False, help="Orbit altitude to use")
    parser.add_argument("--emin", type=float, default=None, help="Minimum elevation (deg)")
    parser.add_argument("--margin-min", type=float, default=0.0, help="Minimum link margin counted (dB)")
    parser.add_argument("--out-csv", required=False, help="Output CSV path")
    args, _ = parser.parse_known_args()

    # If no args given, auto mode (nice for notebooks)
    if len(sys.argv) == 1 or (args.link_csv is None and args.info_csv is None):
        link_csv = find_file("Endurosat_X-band_link_margin_vs_elevation.csv")
        info_csv = find_file("x_down_info.csv")
        if not link_csv or not info_csv:
            raise SystemExit("Auto-mode couldn't find the CSVs. Put them next to the script or pass --link-csv/--info-csv.")
        df = pd.read_csv(link_csv)
        alts = sorted(set(df["orbit_altitude_km"]))
        for alt in alts:
            out_csv = f"pass_data_budget_{int(alt)}km.csv"
            build_budget(link_csv, info_csv, float(alt), args.emin, args.margin_min, out_csv)
        return

    # CLI mode
    if not all([args.link_csv, args.info_csv, args.alt_km, args.out_csv]):
        parser.error("the following arguments are required: --link-csv, --info-csv, --alt-km, --out-csv")

    build_budget(args.link_csv, args.info_csv, args.alt_km, args.emin, args.margin_min, args.out_csv)

if __name__ == "__main__":
    main()


[500 km] time≥thr: 396.9s  data: 3725.6 MiB
[600 km] time≥thr: 422.5s  data: 3181.9 MiB


In [11]:
#!/usr/bin/env python3
"""
Plot pass data budget CSVs produced by make_pass_data_budget.py.

Features
- Auto-detects files named pass_data_budget_*km.csv in the working directory
  (or accept explicit paths via --files).
- For each CSV, creates two figures:
    1) per-bin data contribution vs elevation (MiB per elevation bin)
    2) cumulative data vs elevation (MiB)
- Also creates combined comparison plots overlaying all inputs:
    a) per-bin contributions
    b) cumulative totals
- Saves PNGs next to the inputs (or --outdir).

Usage (auto-discover):
    python plot_pass_data_budgets.py

Usage (explicit files and output dir):
    python plot_pass_data_budgets.py --files pass_data_budget_500km.csv pass_data_budget_600km.csv --outdir plots/

Notes
- Assumes CSV has columns from make_pass_data_budget.py:
  elev_start_deg, elev_end_deg, data_MiB, alt_km, bin_time_s
"""

import argparse
import os
import re
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

RE_ALT = re.compile(r'(\d+)\s*km', re.IGNORECASE)

def parse_alt_from_filename(path: str):
    m = RE_ALT.search(Path(path).name)
    return int(m.group(1)) if m else None

def load_pass_csv(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    required = {"elev_start_deg","elev_end_deg","data_MiB","alt_km","bin_time_s"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"{path} missing columns: {missing}")
    # Sort by elevation start
    df = df.sort_values(["elev_start_deg","elev_end_deg"]).reset_index(drop=True)
    # Add midpoint elevation and cumulative data
    df["elev_mid_deg"] = 0.5*(df["elev_start_deg"] + df["elev_end_deg"])
    df["cum_MiB"] = df["data_MiB"].cumsum()
    # Altitude column might be float; keep an int label for display
    alt_from_file = parse_alt_from_filename(path)
    alt_from_col = None if df.empty else int(round(df["alt_km"].iloc[0]))
    df["alt_label"] = alt_from_file if alt_from_file is not None else alt_from_col
    return df

def find_default_files():
    here = Path(".")
    candidates = list(here.glob("pass_data_budget_*km.csv"))
    if not candidates:
        mnt = Path("/mnt/data")
        candidates = list(mnt.glob("pass_data_budget_*km.csv"))
    return [str(p) for p in candidates]

def plot_per_file(df: pd.DataFrame, out_png_base: str):
    # Per-bin contribution
    plt.figure(figsize=(8,6))
    plt.plot(df["elev_mid_deg"], df["data_MiB"], marker="o")
    plt.title(f"Per-bin Data vs Elevation ({int(df['alt_km'].iloc[0])} km)")
    plt.xlabel("Elevation (degrees)")
    plt.ylabel("Data per bin (MiB)")
    plt.grid(True)
    plt.xticks(sorted(set(df["elev_start_deg"]).union(set(df["elev_end_deg"]))))
    plt.savefig(out_png_base + "_per_bin.png", bbox_inches="tight")
    plt.close()

    # Cumulative
    plt.figure(figsize=(8,6))
    plt.plot(df["elev_end_deg"], df["cum_MiB"], marker="o")
    plt.title(f"Cumulative Data vs Elevation ({int(df['alt_km'].iloc[0])} km)")
    plt.xlabel("Elevation (degrees)")
    plt.ylabel("Cumulative Data (MiB)")
    plt.grid(True)
    plt.xticks(sorted(set(df["elev_start_deg"]).union(set(df["elev_end_deg"]))))
    plt.savefig(out_png_base + "_cumulative.png", bbox_inches="tight")
    plt.close()

def plot_combined(dfs: list[pd.DataFrame], outdir: str):
    # Combined per-bin
    plt.figure(figsize=(8,6))
    for df in dfs:
        label = f"{int(df['alt_km'].iloc[0])} km"
        plt.plot(df["elev_mid_deg"], df["data_MiB"], marker="o", label=label)
    plt.title("Per-bin Data vs Elevation (All Altitudes)")
    plt.xlabel("Elevation (degrees)")
    plt.ylabel("Data per bin (MiB)")
    plt.grid(True)
    # Use union of ticks across all
    ticks = sorted(set().union(*[set(df["elev_start_deg"]).union(set(df["elev_end_deg"])) for df in dfs]))
    plt.xticks(ticks)
    plt.legend()
    plt.savefig(str(Path(outdir)/"pass_data_budget_all_per_bin.png"), bbox_inches="tight")
    plt.close()

    # Combined cumulative
    plt.figure(figsize=(8,6))
    for df in dfs:
        label = f"{int(df['alt_km'].iloc[0])} km"
        plt.plot(df["elev_end_deg"], df["cum_MiB"], marker="o", label=label)
    plt.title("Cumulative Data vs Elevation (All Altitudes)")
    plt.xlabel("Elevation (degrees)")
    plt.ylabel("Cumulative Data (MiB)")
    plt.grid(True)
    ticks = sorted(set().union(*[set(df["elev_start_deg"]).union(set(df["elev_end_deg"])) for df in dfs]))
    plt.xticks(ticks)
    plt.legend()
    plt.savefig(str(Path(outdir)/"pass_data_budget_all_cumulative.png"), bbox_inches="tight")
    plt.close()

def main():
    ap = argparse.ArgumentParser(description="Plot pass data budget CSVs.")
    ap.add_argument("--files", nargs="*", help="Specific CSV files to plot.")
    ap.add_argument("--outdir", default=".", help="Output directory for PNGs.")
    args, _ = ap.parse_known_args()

    files = args.files if args.files else find_default_files()
    if not files:
        raise SystemExit("No pass_data_budget_*km.csv files found. Provide --files or run make_pass_data_budget.py first.")
    os.makedirs(args.outdir, exist_ok=True)

    dfs = []
    for f in files:
        df = load_pass_csv(f)
        dfs.append(df)
        out_base = str(Path(args.outdir)/Path(f).with_suffix("").name)
        plot_per_file(df, out_base)

    if len(dfs) > 1:
        plot_combined(dfs, args.outdir)

    print(f"Wrote plots to: {args.outdir}")
    for f in files:
        base = str(Path(args.outdir)/Path(f).with_suffix("").name)
        print(" -", base + "_per_bin.png")
        print(" -", base + "_cumulative.png")
    if len(dfs) > 1:
        print(" -", str(Path(args.outdir)/"pass_data_budget_all_per_bin.png"))
        print(" -", str(Path(args.outdir)/"pass_data_budget_all_cumulative.png"))

if __name__ == "__main__":
    main()


ValueError: c:\Users\WilliamAvison\AppData\Roaming\jupyter\runtime\kernel-v3c47f55ff71d6f46d6f9e7071bc96acda94032485.json missing columns: {'alt_km', 'data_MiB', 'bin_time_s', 'elev_end_deg', 'elev_start_deg'}

In [12]:
#!/usr/bin/env python3
"""
Notebook/CLI friendly pass data budget generator.

- If run with CLI args, behaves like before.
- If run with *no* args (e.g., pasted into a Jupyter cell), it:
  * looks for Endurosat_X-band_link_margin_vs_elevation.csv and x_down_info.csv
    in ./ or /mnt/data
  * finds all altitudes in the link CSV
  * writes pass_data_budget_<alt>km.csv for each altitude

Outputs now include:
- data_Mb  (megabits, decimal, Mb = 1e6 bits)
- data_MiB (mebibytes, binary, MiB = 1024*1024 bytes)  # kept for compatibility
"""

import argparse, math, sys, os
from pathlib import Path
import pandas as pd

MU_EARTH = 398600.4418  # km^3/s^2
R_EARTH = 6371.0        # km

def find_file(name):
    for p in [Path(name), Path("/mnt/data")/name]:
        if p.is_file():
            return str(p)
    return None

def get_supported_rate(info_csv: str):
    df = pd.read_csv(info_csv)
    s = df.loc[df["python_variable"].astype(str)=="supported_user_data_rate"]
    if s.empty:
        s = df[df["Parameter"].astype(str).str.contains("Supported user data rate", case=False, na=False)]
    if s.empty:
        raise ValueError("Could not find 'Supported user data rate' in x_down_info.csv")
    val = s.iloc[0]["Endurosat X-band"]
    units = s.iloc[0]["Units"] if "Units" in df.columns else ""
    rate_val = float(str(val).strip())
    unit_str = (units or "").lower()
    if "mb" in unit_str:  bps, pretty = rate_val*1e6, "Mb/s"
    elif "kb" in unit_str: bps, pretty = rate_val*1e3, "kb/s"
    elif "b/s" in unit_str or "bps" in unit_str: bps, pretty = rate_val, "b/s"
    else: bps, pretty = rate_val*1e6, "Mb/s (assumed)"
    return bps, pretty

def central_angle_from_slant(range_km: float, alt_km: float) -> float:
    H = R_EARTH + alt_km
    rho = range_km
    cos_theta = (R_EARTH*R_EARTH + H*H - rho*rho) / (2*R_EARTH*H)
    return math.acos(max(-1.0, min(1.0, cos_theta)))

def mean_motion_rad_s(alt_km: float) -> float:
    a = R_EARTH + alt_km
    return math.sqrt(MU_EARTH / (a*a*a))

def build_budget(link_csv: str, info_csv: str, alt_km: float,
                 emin: float | None, margin_min: float, out_csv: str) -> pd.DataFrame:
    df = pd.read_csv(link_csv)
    need = {"orbit_altitude_km","elevation_deg","link_margin_db","slant_range_km"}
    if not need.issubset(df.columns):
        raise ValueError(f"link CSV missing columns: {need - set(df.columns)}")
    dfa = df[df["orbit_altitude_km"].astype(float)==float(alt_km)].copy()
    if dfa.empty:
        raise ValueError(f"No rows for orbit_altitude_km={alt_km}")
    dfa.sort_values("elevation_deg", inplace=True)
    bps, unit_str = get_supported_rate(info_csv)
    if emin is None:
        emin = float(dfa["elevation_deg"].min())

    e = dfa["elevation_deg"].to_list()
    m = dfa["link_margin_db"].to_list()
    r = dfa["slant_range_km"].to_list()
    theta = [central_angle_from_slant(x, alt_km) for x in r]
    n = mean_motion_rad_s(alt_km)

    rows = []
    for i in range(len(e)-1):
        e0, e1 = e[i], e[i+1]
        m0, m1 = m[i], m[i+1]
        th0, th1 = theta[i], theta[i+1]
        if max(e0,e1) < emin: continue
        e_start, e_end = max(min(e0,e1), emin), max(e0,e1)
        if e_end <= e_start: continue

        def lerp(x0,y0,x1,y1,x):
            t=(x-x0)/(x1-x0); return y0+t*(y1-y0)

        if e1 < e0:  # ensure ascending
            e0,e1 = e1,e0; m0,m1 = m1,m0; th0,th1 = th1,th0

        th_start = lerp(e0,th0,e1,th1,e_start)
        th_end   = lerp(e0,th0,e1,th1,e_end)
        m_start  = lerp(e0,m0, e1,m1, e_start)
        m_end    = lerp(e0,m0, e1,m1, e_end)

        dtheta = max(0.0, th_start - th_end)
        bin_time_s = 2.0 * dtheta / n  # rise+set

        usable_fraction = 1.0
        if (m_start < margin_min) and (m_end < margin_min):
            usable_fraction = 0.0
        elif (m_start < margin_min) or (m_end < margin_min):
            if m_end != m_start:
                e_cross = e_start + (margin_min - m_start) * (e_end - e_start) / (m_end - m_start)
                usable_fraction = (e_end - e_cross) / (e_end - e_start) if e_start <= e_cross <= e_end else (1.0 if m_end >= margin_min else 0.0)
            else:
                usable_fraction = 0.0 if m_start < margin_min else 1.0

        data_bits = bps * bin_time_s * usable_fraction
        data_Mb   = data_bits / 1e6                 # decimal megabits
        data_MiB  = data_bits / (1024*1024*8)       # binary mebibytes (kept for compatibility)

        rows.append({
            "alt_km": float(alt_km),
            "elev_start_deg": float(e_start),
            "elev_end_deg": float(e_end),
            "theta_start_deg": math.degrees(th_start),
            "theta_end_deg": math.degrees(th_end),
            "bin_time_s": bin_time_s,
            "margin_start_db": m_start,
            "margin_end_db": m_end,
            "usable_fraction": usable_fraction,
            "data_bits": data_bits,
            "data_Mb": data_Mb,     # NEW: megabits
            "data_MiB": data_MiB,   # legacy column (optional for plots using MiB)
            "rate_bps": bps,
            "rate_unit": unit_str,
        })

    out = pd.DataFrame(rows)
    out.to_csv(out_csv, index=False)
    print(f"[{int(alt_km)} km] time≥thr: {out['bin_time_s'].sum():.1f}s  data: {out['data_Mb'].sum():.1f} Mb")
    return out

def main():
    parser = argparse.ArgumentParser(description="Create data budget for a pass from two CSVs.")
    parser.add_argument("--link-csv", required=False, help="Path to Endurosat_X-band_link_margin_vs_elevation.csv")
    parser.add_argument("--info-csv", required=False, help="Path to x_down_info.csv")
    parser.add_argument("--alt-km", type=float, required=False, help="Orbit altitude to use")
    parser.add_argument("--emin", type=float, default=None, help="Minimum elevation (deg)")
    parser.add_argument("--margin-min", type=float, default=0.0, help="Minimum link margin counted (dB)")
    parser.add_argument("--out-csv", required=False, help="Output CSV path")
    args, _ = parser.parse_known_args()

    # If no args given, auto mode (nice for notebooks)
    if len(sys.argv) == 1 or (args.link_csv is None and args.info_csv is None):
        link_csv = find_file("Endurosat_X-band_link_margin_vs_elevation.csv")
        info_csv = find_file("x_down_info.csv")
        if not link_csv or not info_csv:
            raise SystemExit("Auto-mode couldn't find the CSVs. Put them next to the script or pass --link-csv/--info-csv.")
        df = pd.read_csv(link_csv)
        alts = sorted(set(df["orbit_altitude_km"]))
        for alt in alts:
            out_csv = f"pass_data_budget_{int(alt)}km.csv"
            build_budget(link_csv, info_csv, float(alt), args.emin, args.margin_min, out_csv)
        return

    # CLI mode
    if not all([args.link_csv, args.info_csv, args.alt_km, args.out_csv]):
        parser.error("the following arguments are required: --link-csv, --info-csv, --alt-km, --out-csv")

    build_budget(args.link_csv, args.info_csv, args.alt_km, args.emin, args.margin_min, args.out_csv)

if __name__ == "__main__":
    main()


[500 km] time≥thr: 396.9s  data: 31252.3 Mb
[600 km] time≥thr: 422.5s  data: 26691.8 Mb


scale to 600s pass

In [14]:
# Notebook-friendly: scale pass_data_budget_*km.csv to a target duration
# and produce scaled CSVs + a combined cumulative PNG.

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# --- Config you can tweak ---
TARGET_DURATION_S = 600.0   # 10 minutes
OUTDIR = Path(".")          # where to save outputs
FILES = None                # e.g. ["pass_data_budget_500km.csv", "pass_data_budget_600km.csv"]; or leave None to auto-find
# ----------------------------

def find_pass_files():
    pats = [Path(".").glob("pass_data_budget_*km.csv"),
            Path("/mnt/data").glob("pass_data_budget_*km.csv")]
    files = []
    for g in pats:
        for p in g:
            # guard: only CSVs with our naming pattern
            if p.is_file() and p.suffix.lower() == ".csv" and "pass_data_budget_" in p.name:
                files.append(p.resolve())
    # de-dup while preserving order
    seen, unique = set(), []
    for p in files:
        if p not in seen:
            unique.append(p); seen.add(p)
    return unique

def scale_one(path: Path, duration_s: float):
    df = pd.read_csv(path)
    required = {"bin_time_s", "rate_bps", "usable_fraction", "alt_km"}
    missing = required - set(df.columns)
    if missing:
        print(f"⚠️  Skipping {path.name}: missing columns {missing}")
        return None, None

    base_time = float(df["bin_time_s"].sum())
    if base_time <= 0:
        print(f"⚠️  Skipping {path.name}: total bin_time_s is zero")
        return None, None

    scale = duration_s / base_time
    d = df.copy()
    d["bin_time_s_scaled"] = d["bin_time_s"] * scale
    d["data_bits_scaled"] = d["rate_bps"] * d["bin_time_s_scaled"] * d["usable_fraction"]
    d["data_Mb_scaled"] = d["data_bits_scaled"] / 1e6            # decimal megabits
    d["data_MiB_scaled"] = d["data_bits_scaled"] / (1024**2 * 8) # binary mebibytes (optional)
    d["cum_time_s_scaled"] = d["bin_time_s_scaled"].cumsum()
    d["cum_Mb_scaled"] = d["data_Mb_scaled"].cumsum()
    alt = int(round(d["alt_km"].iloc[0]))
    summary = {
        "file": path.name,
        "alt_km": alt,
        "base_time_s": base_time,
        "scaled_time_s": float(d["bin_time_s_scaled"].sum()),
        "scaled_Mb": float(d["data_Mb_scaled"].sum()),
        "scale_factor": scale,
    }
    return d, summary

# Discover files
paths = [Path(p) for p in FILES] if FILES else find_pass_files()
if not paths:
    raise SystemExit("No pass_data_budget_*km.csv files found. Run your budget generator first.")

OUTDIR.mkdir(parents=True, exist_ok=True)

scaled_list, summaries = [], []
for p in paths:
    d, s = scale_one(p, TARGET_DURATION_S)
    if d is None: 
        continue
    minutes = int(round(TARGET_DURATION_S / 60))
    alt = s["alt_km"]
    out_csv = OUTDIR / f"pass_data_budget_{alt}km_scaled_{minutes}min.csv"
    d.to_csv(out_csv, index=False)
    scaled_list.append(d)
    summaries.append(s)
    print(f"[{alt} km] base {s['base_time_s']:.1f}s → scaled {s['scaled_time_s']:.1f}s | "
          f"data {s['scaled_Mb']:.1f} Mb → {out_csv.name}")

# Plot combined cumulative Mb vs time
if scaled_list:
    plt.figure(figsize=(8,6))
    for d in scaled_list:
        alt = int(round(d["alt_km"].iloc[0]))
        plt.plot(d["cum_time_s_scaled"], d["cum_Mb_scaled"], marker="o", label=f"{alt} km")
    plt.title(f"Cumulative Data vs Time (Scaled to {int(round(TARGET_DURATION_S/60))}-minute Pass)")
    plt.xlabel("Time (s)")
    plt.ylabel("Cumulative Data (Mb)")
    plt.grid(True)
    plt.legend()
    png_path = OUTDIR / f"pass_data_budget_scaled_{int(round(TARGET_DURATION_S/60))}min_cumulative.png"
    plt.savefig(png_path, bbox_inches="tight")
    plt.close()
    print(f"Wrote cumulative plot → {png_path}")

# Show a quick summary table in the notebook (optional)
pd.DataFrame(summaries)


[500 km] base 396.9s → scaled 600.0s | data 47243.9 Mb → pass_data_budget_500km_scaled_10min.csv
[600 km] base 422.5s → scaled 600.0s | data 37905.6 Mb → pass_data_budget_600km_scaled_10min.csv
Wrote cumulative plot → pass_data_budget_scaled_10min_cumulative.png


,file,alt_km,base_time_s,scaled_time_s,scaled_Mb,scale_factor
0,pass_data_budget_500km.csv,500,396.905248,600.0,47243.936120,1.511696
1,pass_data_budget_600km.csv,600,422.499347,600.0,37905.572941,1.420121


In [15]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# --- Config ---
TARGET_DURATION_S = 600.0   # 10 minutes
OUTDIR = Path(".")
FILES = None                # or like ["pass_data_budget_500km.csv", ...]
# ---------------

def find_pass_files():
    pats = [Path(".").glob("pass_data_budget_*km.csv"),
            Path("/mnt/data").glob("pass_data_budget_*km.csv")]
    files = []
    for g in pats:
        for p in g:
            if p.is_file() and p.suffix.lower()==".csv" and "pass_data_budget_" in p.name:
                files.append(p.resolve())
    seen, uniq = set(), []
    for p in files:
        if p not in seen:
            uniq.append(p); seen.add(p)
    return uniq

def scale_one(path: Path, duration_s: float):
    df = pd.read_csv(path)
    required = {"bin_time_s", "rate_bps", "usable_fraction", "alt_km"}
    missing = required - set(df.columns)
    if missing:
        print(f"⚠️  Skipping {path.name}: missing {missing}")
        return None, None

    base_time = float(df["bin_time_s"].sum())
    if base_time <= 0:
        print(f"⚠️  Skipping {path.name}: total bin_time_s is zero")
        return None, None

    scale = duration_s / base_time
    d = df.copy()
    d["bin_time_s_scaled"] = d["bin_time_s"] * scale
    d["data_bits_scaled"]  = d["rate_bps"] * d["bin_time_s_scaled"] * d["usable_fraction"]

    # Megabytes (decimal) and Mebibytes (binary)
    d["data_MB_scaled"]  = d["data_bits_scaled"] / (8 * 1e6)
    d["data_MiB_scaled"] = d["data_bits_scaled"] / (8 * 1024**2)

    # Cumulative for plotting
    d["cum_time_s_scaled"] = d["bin_time_s_scaled"].cumsum()
    d["cum_MB_scaled"]     = d["data_MB_scaled"].cumsum()

    alt = int(round(d["alt_km"].iloc[0]))
    summary = {
        "file": path.name,
        "alt_km": alt,
        "base_time_s": base_time,
        "scaled_time_s": float(d["bin_time_s_scaled"].sum()),
        "scaled_MB": float(d["data_MB_scaled"].sum()),
        "scale_factor": scale,
    }
    return d, summary

paths = [Path(p) for p in FILES] if FILES else find_pass_files()
if not paths:
    raise SystemExit("No pass_data_budget_*km.csv files found.")

OUTDIR.mkdir(parents=True, exist_ok=True)

scaled_list, summaries = [], []
for p in paths:
    d, s = scale_one(p, TARGET_DURATION_S)
    if d is None: 
        continue
    minutes = int(round(TARGET_DURATION_S/60))
    alt = s["alt_km"]
    out_csv = OUTDIR / f"pass_data_budget_{alt}km_scaled_{minutes}min.csv"
    d.to_csv(out_csv, index=False)
    scaled_list.append(d)
    summaries.append(s)
    print(f"[{alt} km] base {s['base_time_s']:.1f}s → scaled {s['scaled_time_s']:.1f}s | "
          f"data {s['scaled_MB']:.1f} MB → {out_csv.name}")

# Combined cumulative MB vs time
if scaled_list:
    plt.figure(figsize=(8,6))
    for d in scaled_list:
        alt = int(round(d["alt_km"].iloc[0]))
        plt.plot(d["cum_time_s_scaled"], d["cum_MB_scaled"], marker="o", label=f"{alt} km")
    plt.title(f"Cumulative Data vs Time (Scaled to {int(round(TARGET_DURATION_S/60))}-minute Pass)")
    plt.xlabel("Time (s)")
    plt.ylabel("Cumulative Data (MB)")
    plt.grid(True)
    plt.legend()
    png_path = OUTDIR / f"pass_data_budget_scaled_{int(round(TARGET_DURATION_S/60))}min_cumulative.png"
    plt.savefig(png_path, bbox_inches="tight")
    plt.close()
    print(f"Wrote cumulative plot → {png_path}")

pd.DataFrame(summaries)


[500 km] base 396.9s → scaled 600.0s | data 5905.5 MB → pass_data_budget_500km_scaled_10min.csv
[600 km] base 422.5s → scaled 600.0s | data 4738.2 MB → pass_data_budget_600km_scaled_10min.csv
Wrote cumulative plot → pass_data_budget_scaled_10min_cumulative.png


,file,alt_km,base_time_s,scaled_time_s,scaled_MB,scale_factor
0,pass_data_budget_500km.csv,500,396.905248,600.0,5905.492015,1.511696
1,pass_data_budget_600km.csv,600,422.499347,600.0,4738.196618,1.420121
